## Euler's integration

In [ ]:
# Init simulation

import pygame
from matplotlib import pyplot as plt

from physics import kinetic, universe, universe_utils
from utils.vector import Vector
from graphics import manager

pygame.display.set_mode = lambda *_: None

mgr = manager.Manager()

TICKS = 100
DT = 10

uni = universe.Universe()

In [ ]:
sun_r = 6.96e5
base_point = Vector(820 * universe_utils.UNIT_SIZE, 540 * universe_utils.UNIT_SIZE)

sun = kinetic.Kinetic(
    1.989e30,
    base_point,
    (255, 255, 255),
    sun_r,
    0,
    "Sun"
)

sun_g = universe_utils.G_CONST * sun.mass / (sun.astro_radius ** 2)
uni.kinetic_registry.register(sun)

earth_pos = base_point + Vector(1, 0) * 1.49597e8
earth = kinetic.Kinetic(5.97e24, earth_pos, (255, 255, 255), 6378, 1, 'Earth')
earth.apply_velocity(universe_utils.generate_v1(earth, sun))

uni.kinetic_registry.register(earth)

x = []
y = []

ax = []
ay = []

dr = []

E = []

index = 0

for tick in range(TICKS):
    x.append(DT * tick)
    y.append(earth.current_velocity.magnitude)

    ax.append(earth.current_velocity.x)
    ay.append(earth.current_velocity.y)
    Ek = (earth.current_velocity.magnitude ** 2) / 2
    Ep = universe_utils.distance(earth, sun)
    E.append(Ek + Ep)
    dr.append(universe_utils.distance(earth, sun))
    uni.tick(DT)

In [ ]:
err = [ (energy / E[0] - 1) * 100 for energy in E ]
plt.plot(x, err)
plt.title("System energy error over time")

plt.xlabel('Time')
plt.ylabel('drift, %')
plt.show()

# Plotting data    
plt.plot(x, y)
plt.plot(x, ax)
plt.plot(x, ay)
plt.xlabel('Time')
plt.ylabel('Velocity (magnitude, x, y)')

plt.title('Earth velocity over time (magnitude, velocity.x, velocity.y)')

plt.show()

plt.plot(x, dr)

aphelion = dr.index(max(dr))
perihelion = dr.index(min(dr))

plt.plot(x[aphelion], dr[aphelion], 'ro')
plt.annotate(str(dr[aphelion] / 1e6), (x[aphelion], dr[aphelion]))
plt.plot(x[perihelion], dr[perihelion], 'ro')
plt.annotate(str(dr[perihelion] / 1e6), (x[perihelion], dr[perihelion]))
plt.title('Earth orbit params (aphelion, perihelion), m')
plt.show()

In [ ]:
x = []
y = []
dr = []

for tick in range(TICKS):
    x.append(DT)
    y.append(earth.current_velocity.magnitude)
    dr.append(universe_utils.distance(earth, sun))
    uni.tick(DT)
    DT += 1

# Plotting data    
plt.plot(x, y)
plt.xlabel('Time')
plt.ylabel('Velocity (magnitude)')

plt.title('Earth velocity over time (magnitude) with tick drift')

plt.show()

plt.plot(x, dr)

aphelion = dr.index(max(dr))
perihelion = dr.index(min(dr))

plt.plot(x[aphelion], dr[aphelion], 'ro')
plt.annotate(str(dr[aphelion] / 1e6), (x[aphelion], dr[aphelion]))
plt.plot(x[perihelion], dr[perihelion], 'ro')
plt.annotate(str(dr[perihelion] / 1e6), (x[perihelion], dr[perihelion]))
plt.title('Earth orbit params (aphelion, perihelion), m')
plt.show()

In [ ]:
uni.finalize()

# Cleanup all kinetics, restart the universe